
```
---
title: API Review with Foursquare API
type:  lesson + lab + demo
duration: "1:25"
creator:
    name: David Yerrington
    city: SF
---
```
<img src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" style="float: left; margin: 10px">

#  API Demo / Lab + NLP
Week 8 | 3.3


<img src="https://snag.gy/RNAEgP.jpg" width="600">

Can we correctly identify which of these two old men tweeted what?


## (5 mins) Opening 

Today we are going to attempt to classify wether a tweet comes from Trump, or Sanders.  We are going to:

- Create a developer account on Twitter
- Create a method to pull a list of tweets from the Twitter API
- Perform proper preprocessing on our text
- Engineer sentiment feature in our dataset using TextBlob
- Explore supervised classification techniques


## Twitter API Developer Registration

If you haven't registered a Twitter account yet, this is a requirement in order to have a "developer" account.

[Twitter Rest API](https://dev.twitter.com/rest/public)



## Create an "App"

![](https://snag.gy/HPBQbJ.jpg)

We now will now go to Twitter and register an "app" [apps.twitter.com](https://apps.twitter.com/), just like we did for Foursquare.  After we set up our app, we will only need to reference the cooresponding keys Twitter generates for our app.  These are the keys that we will use with our application to communicate with the Twitter API.

## Install Python Twitter API library

Someone was nice enough to build a nice libary for us in Python that we only need to plug in our keys and start collecting data with.  The library we will be using is provided by [Python Twitter Tools](http://mike.verdone.ca/twitter/).

To install it, just run the next frame (there is no conda package).

In [1]:
!pip install twitter python-twitter

    100% |████████████████████████████████| 61kB 785kB/s 
    100% |████████████████████████████████| 81kB 1.1MB/s 
    100% |████████████████████████████████| 112kB 1.2MB/s 
  Running setup.py bdist_wheel for python-twitter ... - done
  Stored in directory: /Users/sebozek/Library/Caches/pip/wheels/22/1e/2e/506871fa7dc610616948e70812d5e2518cd89c13f757b98f6c
  Running setup.py bdist_wheel for oauthlib ... - done
  Stored in directory: /Users/sebozek/Library/Caches/pip/wheels/e6/be/43/e4a2ca8cb9c78fbd9b5b14b96cb7a5cc43f36bc11af5dfac5b
Successfully built python-twitter oauthlib


## Some Boring Twitter Rules

Twitter says they will rate limit your requests:

>When using application-only authentication, rate limits are determined globally for the entire application. If a method allows for 15 requests per rate limit window, then it allows you to make 15 requests per window — on behalf of your application. This limit is considered completely separately from per-user limits. https://dev.twitter.com/rest/public/rate-limiting

Here's a quick overview of what Twitter says are "the rulez":

![](https://snag.gy/yJ6vIH.jpg)


## About those Keys: OAuth Review

![](https://g.twimg.com/dev/documentation/image/appauth_0.png)

## What's going on here?  Take a minute..

## Our Application Keys

Take note of our application keys that we will be using with our little application that will be connecting to Twitter and mining Tweets from the official Bernie Sanders and Donald Trump twitter accounts.

![](https://snag.gy/H1djQK.jpg)

## Tweet Miner Class Setup

The following code is meant to get us up and running with connectivity to twitter, and the ability to make requests and easily transform the JSON responses to DataFrames.  We will be using object oriented Python in order to organize our code.  We may go into review since this was a topic we covered earlier in the class but we can review it during the lab for those who want to know more about it.


> "request_limit" is used in this class to limit the number of tweets that are pulled per instance request.  Setting it to something lower until you've worked the bugs out of your request, and captured the data you want, is essential to avoiding any rate limit blocks.

#### Key Setup

- **consumer_key** - Find this in your app page under the "Keys and Access Tokens"
- **consumer_secret** - Right under **consumer_key** in the "Keys and Access Tokens" tab
- **access_token_key** - You will need to click the "generate tokens" button to get this
- **access_token_secret** - Also available after "generate tokens" is pressed


In [9]:
import twitter, re, datetime, pandas as pd

class twitterminer():

    request_limit   =   20    
    api             =   False
    data            =   []
    
    twitter_keys = {
        'consumer_key':        'qvWLs0mSiw9tRQWRLCmXE412N',
        'consumer_secret':     'r9hGRjzgToCmBHaasgUDiImqjqS65kP7opTnXYUwDoVYoUbRi3',
        'access_token_key':    '1887369426-X1nL75QFZA3PoK9NZqJSlSAqUYimfzav08uGyA2',
        'access_token_secret': 'FlxLFVghY5ljqHgwoYg3ueSpuOudUZ8RaQXirusRUcTft'
    }
    
    def __init__(self,  request_limit = 20):
        
        self.request_limit = request_limit
        
        # This sets the twitter API object for use internall within the class
        self.set_api()
        
    def set_api(self):
        
        self.api = twitter.Api(
            consumer_key         =   self.twitter_keys['consumer_key'],
            consumer_secret      =   self.twitter_keys['consumer_secret'],
            access_token_key     =   self.twitter_keys['access_token_key'],
            access_token_secret  =   self.twitter_keys['access_token_secret']
        )

    def mine_user_tweets(self, user="dyerrington", mine_rewteets=False):

        statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.request_limit)
        data       =   []
        
        for item in statuses:

            mined = {
                'tweet_id': item.id,
                'handle': item.user.name,
                'retweet_count': item.retweet_count,
                'text': item.text,
                'mined_at': datetime.datetime.now(),
                'created_at': item.created_at,
            }
            
            data.append(mined)
            
        return data

## Does anyone remember how we "instantiate" a new instance of this class?

**Bonus bonus** How do we call the method to *mine_user_tweets()*?

In [54]:
# twitter ids:  realDonaldTrump, berniesanders
# Let's test this out here..

twit = twitterminer() # Instantiate a new instance of the class

twit.mine_user_tweets(user='realDonaldTrump')

[{'created_at': u'Wed Jun 01 13:17:58 +0000 2016',
  'handle': u'Donald J. Trump',
  'mined_at': datetime.datetime(2016, 6, 1, 11, 0, 5, 52157),
  'retweet_count': 5415,
  'text': u'So I raised/gave $5,600,000 for the veterans and the media makes me look bad! They do anything to belittle - totally biased.',
  'tweet_id': 737996614573260800},
 {'created_at': u'Tue May 31 23:18:28 +0000 2016',
  'handle': u'Donald J. Trump',
  'mined_at': datetime.datetime(2016, 6, 1, 11, 0, 5, 52167),
  'retweet_count': 4049,
  'text': u'Congratulations to @seanhannity on his tremendous increase in television ratings. Speaking of ratings- I will be on his show tonight @ 10pE.',
  'tweet_id': 737785345459113985},
 {'created_at': u'Tue May 31 23:17:33 +0000 2016',
  'handle': u'Donald J. Trump',
  'mined_at': datetime.datetime(2016, 6, 1, 11, 0, 5, 52169),
  'retweet_count': 5895,
  'text': u'I am getting great credit for my press conference today. Crooked Hillary should be admonished for not having a pre

##  Now we create some training data

We will have to munge a little bit in order to get our "mined" data from the Twitter API.  

 - Mine Trump Tweets
 - Create DataFrame
 - Mine Sanders Tweets
 - Append to DataFrame

In [55]:
# we only need to "instantiate" once.  Then we can call mine_user_tweets as much as we want.
miner = twitterminer(request_limit=100)
trump_tweets = miner.mine_user_tweets("realDonaldTrump")

In [67]:
trump_df = pd.DataFrame(trump_tweets)
trump_df.head(10)

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Wed Jun 01 13:17:58 +0000 2016,Donald J. Trump,2016-06-01 11:00:41.884034,5422,"So I raised/gave $5,600,000 for the veterans a...",737996614573260800
1,Tue May 31 23:18:28 +0000 2016,Donald J. Trump,2016-06-01 11:00:41.884051,4049,Congratulations to @seanhannity on his tremend...,737785345459113985
2,Tue May 31 23:17:33 +0000 2016,Donald J. Trump,2016-06-01 11:00:41.884055,5895,I am getting great credit for my press confere...,737785114239864832
3,Tue May 31 23:14:42 +0000 2016,Donald J. Trump,2016-06-01 11:00:41.884058,5777,"Katie Couric, the third rate reporter, who has...",737784398515441664
4,Tue May 31 22:51:56 +0000 2016,Donald J. Trump,2016-06-01 11:00:41.884060,2140,I will be interviewed on @seanhannity tonight ...,737778668785766404
5,Tue May 31 21:12:07 +0000 2016,Donald J. Trump,2016-06-01 11:00:41.884064,2934,"Join me in Sacramento, California-tomorrow eve...",737753549564694528
6,Tue May 31 16:57:17 +0000 2016,Donald J. Trump,2016-06-01 11:00:41.884067,5156,So many veterans groups are beyond happy with ...,737689416811073536
7,Tue May 31 16:53:58 +0000 2016,Donald J. Trump,2016-06-01 11:00:41.884070,5225,Just finished a press conference in Trump Towe...,737688582849171456
8,Tue May 31 11:42:20 +0000 2016,Donald J. Trump,2016-06-01 11:00:41.884074,5280,I have raised/given a tremendous amount of mon...,737610159459991552
9,Tue May 31 10:48:49 +0000 2016,Donald J. Trump,2016-06-01 11:00:41.884079,3903,So many great things happening - new poll numb...,737596690257969152


## Any interesting ngrams going on with Trump?

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

# With Stop Words

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer( ngram_range=(2,2) )

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(10)

[(u'https co', 31),
 (u'thank you', 14),
 (u'crooked hillary', 10),
 (u'of the', 10),
 (u'trump2016 https', 8),
 (u'in the', 8),
 (u'for the', 6),
 (u'will be', 6),
 (u'great again', 5),
 (u'america great', 5)]

In [69]:
# Without Stop Words

# We can use the TfidfVectorizer to find ngrams for us
vect = TfidfVectorizer(stop_words='english', ngram_range=(2,2))

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(10)

[(u'crooked hillary', 10),
 (u'trump2016 https', 8),
 (u'make america', 5),
 (u'america great', 5),
 (u'rolling thunder', 5),
 (u'hillary clinton', 5),
 (u'elizabeth warren', 4),
 (u'memorial day', 3),
 (u'totally biased', 3),
 (u'dishonest media', 3)]

## (10 mins) Try this exercize with Bernie Sanders..

In [80]:
sanders_tweets = miner.mine_user_tweets("berniesanders")

In [81]:
sanders_df = pd.DataFrame(sanders_tweets)
sanders_df.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Wed Jun 01 18:04:31 +0000 2016,Bernie Sanders,2016-06-01 11:32:20.962080,361,We need to implement a nationwide ban on frack...,738068724356370432
1,Wed Jun 01 17:07:42 +0000 2016,Bernie Sanders,2016-06-01 11:32:20.962091,348,We need to put an end to fracking all over thi...,738054427341402114
2,Wed Jun 01 16:41:55 +0000 2016,Bernie Sanders,2016-06-01 11:32:20.962094,775,If I am elected president we will work togethe...,738047938681704450
3,Wed Jun 01 16:24:02 +0000 2016,Bernie Sanders,2016-06-01 11:32:20.962097,898,Secretary Clinton and I have differences of op...,738043438021500928
4,Wed Jun 01 14:58:07 +0000 2016,Bernie Sanders,2016-06-01 11:32:20.962099,1061,"When we pass Medicare for all, everyone will h...",738021814983491584


In [82]:
vect = TfidfVectorizer(
                       ngram_range=(2,2)
)

In [83]:
# make summary of the documents
summary = "".join(sanders_df['text'])

ngrams_summaries = vect.fit(sanders['text']).build_analyzer()(summary)

Counter(ngrams_summaries).most_common(10)

[(u'https co', 50),
 (u'need to', 8),
 (u'of the', 8),
 (u'this country', 6),
 (u'in the', 5),
 (u'we need', 5),
 (u'here https', 5),
 (u'tune in', 5),
 (u'on june', 4),
 (u'we are', 4)]

In [84]:
vect = TfidfVectorizer(ngram_range=(2,2),
                       stop_words='english')

In [85]:
ngrams_summaries = vect.fit(sanders_df['text']).build_analyzer()(summary)

In [86]:
Counter(ngrams_summaries).most_common(10)

[(u'climate change', 4),
 (u'bernie bernie', 4),
 (u'bernie sanders', 4),
 (u'health care', 4),
 (u'june https', 3),
 (u'wall street', 3),
 (u'social conversation', 3),
 (u'donald trump', 3),
 (u'live https', 3),
 (u'final fec', 2)]

In [87]:
all_tweets = pd.DataFrame(trump_tweets + sanders_tweets)

## Preprocessing our Tweets

In order to do classfication recall that we need a set of features.  Our features are literally what our presidential hopefulls say on Twitter. 

We will need to:
- Vectorize input text data
- Intialize a model (let's try Logistic regression)
- Train / Predict / Cross Validate
- Score / Evaluate


In [88]:
from sklearn.linear_model import LogisticRegression

# Preprocess our text data to Tfidf
tfv = TfidfVectorizer(lowercase=True, strip_accents='unicode')
X_all = tfv.fit_transform(all_tweets['text'])

# Setup logistic regression (or try another classification method here)
estimator = LogisticRegression()
estimator.fit(X_all, all_tweets['handle'])



LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Check Prediction vs Random Sanders Tweet

In [95]:
# Prep our source as TfIdf vectors
source_test = [
    "Demanding that the wealthy and the powerful start paying their fair share of taxes that's exactly what the American people want.",
    "Crooked Hillary is spending tremendous amounts of Wall Street money on false ads against me. She is a very dishonest person!",
]


############
# NOTE:  Do not re-initialize the tfidf vectorizor or the feature space willbe overwritten and
# hence your transform will not match the number of features you trained your model on.
#
# This is why you only need to "transform" since you already "fit" previously
#
####

X_all = tfv.transform(source_test)

# Predict using previously trained logist regression `estimator`
print estimator.predict_proba(X_all)
print estimator.predict(X_all)

[[ 0.64385463  0.35614537]
 [ 0.29054443  0.70945557]]
[u'Bernie Sanders' u'Donald J. Trump']


## Lab Time

We would like you to perform an analysis using a proper cross validation.  Also, try classfication using other models.

### 1. Implement the same analysis using more data.

Experiment with using more data.  The API may not like that you are blowing through their limits so definitely be careful.  Try to grab only what you need 1x, then work on the copy of the objects that are returned.  Read the documents about rate limits and see if you can get enough without hitting the rate limit.  Are there any options availabl in the API to avoid such a problem?

In [96]:
# we only need to "instantiate" once.  Then we can call mine_user_tweets as much as we want.
miner = twitterminer(request_limit=200)
trump_tweets = miner.mine_user_tweets("realDonaldTrump")

In [99]:
miner = twitterminer(request_limit=200)
bernie_tweets = miner.mine_user_tweets("berniesanders")

In [107]:
all_tweets = pd.DataFrame(trump_tweets + bernie_tweets)

In [108]:
all_tweets.shape

(400, 6)

In [109]:
bern_df = pd.DataFrame(bernie_tweets)
trump_df = pd.DataFrame(trump_tweets)

## Bernie

In [110]:
vect = TfidfVectorizer(ngram_range=(2,2), stop_words='english')

In [113]:
bern_n = vect.fit(bern_df.text).build_analyzer()("".join(bern_df.text))

Counter(bern_n).most_common(10)

[(u'political revolution', 9),
 (u'puerto rico', 9),
 (u'vote bernie', 7),
 (u'health care', 5),
 (u'wall street', 5),
 (u'donald trump', 5),
 (u'climate change', 5),
 (u'california https', 4),
 (u'bernie bernie', 4),
 (u'fossil fuel', 4)]

## Trump

In [114]:
trump_n = vect.fit(trump_df.text).build_analyzer()("".join(trump_df.text))

Counter(trump_n).most_common(10)

[(u'crooked hillary', 26),
 (u'hillary clinton', 13),
 (u'trump2016 https', 11),
 (u'megynkelly realdonaldtrump', 11),
 (u'failing nytimes', 7),
 (u'america great', 6),
 (u'make america', 6),
 (u'rolling thunder', 5),
 (u'close deal', 4),
 (u'rowanne brewer', 4)]

### 2. Implement K-Folds or test/train split.

Double check that you are getting random data before moving forward.  What would happen if you over sample Trump more than Sanders?

### 3. Mine more Tweets that aren't in your data set
Or use the hold-out method to do a proper test.  Refer back to our advanced classification evaluation lesson if you need to.

### 4. Check your classification report
How's precision / recall of your model?

### 5.  Change out your TFIDF vectorizer for CountVectorizer.
How has this impacted your mode performance at all?

### 6.  Implement a different classification method such as random forrests.
Or pick one of your favorites

### 7.  Try to remove stopwords from your text during your preprocessing step

Then double check your classfication report.  Have things improved?

### 8.  Try removing samples that have links or that are obviously just announcements or "noise" that doesn't appear to represent "True" tweets by the authors.

### 9. What are some contrasting words or phrases that you can see between the ngrams for each author?

### 10.  What do you think you can do to improve the scores further?

### 11. **BONUS** Using TextBlob, add a sentiment feature to your dataset.

In [120]:
from textblob import TextBlob

In [136]:
# all_tweets['Sentiment'] = TextBlob(all_tweets['text']).sentiment

all_tweets['Polarity'] = all_tweets['text'].apply(lambda x : TextBlob(x).sentiment.polarity)
all_tweets['Subjectivity'] = all_tweets['text'].apply(lambda x : TextBlob(x).sentiment.subjectivity)

In [138]:
# del all_tweets['Sentiment']

In [139]:
all_tweets.head()

,created_at,handle,mined_at,retweet_count,text,tweet_id,Polarity,Subjectivity
0,Wed Jun 01 13:17:58 +0000 2016,Donald J. Trump,2016-06-01 11:58:19.858409,5854,"So I raised/gave $5,600,000 for the veterans a...",737996614573260800,-0.437500,0.708333
1,Tue May 31 23:18:28 +0000 2016,Donald J. Trump,2016-06-01 11:58:19.858421,4108,Congratulations to @seanhannity on his tremend...,737785345459113985,0.333333,1.000000
2,Tue May 31 23:17:33 +0000 2016,Donald J. Trump,2016-06-01 11:58:19.858424,5960,I am getting great credit for my press confere...,737785114239864832,0.400000,0.425000
3,Tue May 31 23:14:42 +0000 2016,Donald J. Trump,2016-06-01 11:58:19.858426,5870,"Katie Couric, the third rate reporter, who has...",737784398515441664,0.107143,0.214286
4,Tue May 31 22:51:56 +0000 2016,Donald J. Trump,2016-06-01 11:58:19.858428,2151,I will be interviewed on @seanhannity tonight ...,737778668785766404,0.500000,0.500000


### 12. BONUS BONUS Apply PCA to your text features
Is this effective? (ie: we could talk about LDA here a little bit)

## Closing

- What where the most impactful changes that helped your models?
- What do you think would happen if we had more Trump Tweets than Sanders?
- What other projects might you think to apply these problems against?